In [1]:
with open('./data/Sonnet.txt') as f:
    sonnet = f.read()

result = sonnet.split("\n")
result

['ï»¿From fairest creatures we desire increase,',
 'That thereby beautyâ€™s rose might never die,',
 'But as the riper should by time decease,',
 'His tender heir might bear his memory:',
 'But thou, contracted to thine own bright eyes,',
 'Feedâ€™st thy lightâ€™s flame with self-substantial fuel,',
 'Making a famine where abundance lies,',
 'Thyself thy foe, to thy sweet self too cruel:',
 'Thou that art now the worldâ€™s fresh ornament,',
 'And only herald to the gaudy spring,',
 'Within thine own bud buriest thy content,',
 'And tender churl makâ€™st waste in niggarding:',
 'Pity the world, or else this glutton be,',
 'To eat the worldâ€™s due, by the grave and thee.',
 '',
 'When forty winters shall besiege thy brow,',
 'And dig deep trenches in thy beautyâ€™s field,',
 'Thy youthâ€™s proud livery so gazed on now,',
 'Will be a tatterâ€™d weed of small worth held:',
 'Then being asked, where all thy beauty lies,',
 'Where all the treasure of thy lusty days;',
 'To say, within thine

In [2]:
# Splitting each sonnet into a list of lines
sonnets = sonnet.split("\n\n")

In [26]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [4]:
import pandas as pd

sonnets = pd.DataFrame(sonnets)
sonnets

,0
0,"ï»¿From fairest creatures we desire increase,\..."
1,"When forty winters shall besiege thy brow,\nAn..."
2,Look in thy glass and tell the face thou viewe...
3,
4,"Unthrifty loveliness, why dost thou spend\nUpo..."
...,...
280,"In loving thee thou knowâ€™st I am forsworn,\n..."
281,
282,Cupid laid by his brand and fell asleep:\nA ma...
283,


In [5]:
valSonnets = sonnets.values

# Remove empty lines
valSonnets = [x for x in valSonnets if x != '']
valSonnets = pd.DataFrame(valSonnets)

In [15]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=1000, oov_token='<OOV>'
)

tokenizer.fit_on_texts(valSonnets[0])

# Tokenize the text
sequences = tokenizer.texts_to_sequences(valSonnets[0])

# Create a word index
word_index = tokenizer.word_index

# Pad sequences
padded = tf.keras.preprocessing.sequence.pad_sequences(sequences, padding='post',
                                                       truncating='post')

In [22]:
len(padded[0])

130

In [23]:
TOTAL_WORDS, MAX_SEQ = 1000, 130
    
model = tf.keras.Sequential()
### START CODE HERE
model.add(tf.keras.layers.Embedding(TOTAL_WORDS, 100, input_length=MAX_SEQ-1))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150)))
model.add(tf.keras.layers.Dense(TOTAL_WORDS, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
     

In [24]:
def features_and_labels(input_sequences, total_words):
    features = input_sequences[:,:-1]
    labels = input_sequences[:,-1]
    one_hot_labels = tf.keras.utils.to_categorical(labels, num_classes=total_words)
    return features, one_hot_labels

features, labels = features_and_labels(padded, TOTAL_WORDS)

In [25]:
history = model.fit(features, labels, epochs=50, verbose=1)

Epoch 1/50
5/5 [==============================] - 9s 1s/step - loss: 6.8161 - accuracy: 0.7922
Epoch 2/50
5/5 [==============================] - 6s 1s/step - loss: 5.4736 - accuracy: 0.9935
Epoch 3/50
5/5 [==============================] - 6s 1s/step - loss: 0.7756 - accuracy: 0.9935
Epoch 4/50
5/5 [==============================] - 6s 1s/step - loss: 0.0751 - accuracy: 0.9935
Epoch 5/50
5/5 [==============================] - 6s 1s/step - loss: 0.0591 - accuracy: 0.9935
Epoch 6/50
5/5 [==============================] - 6s 1s/step - loss: 0.0587 - accuracy: 0.9935
Epoch 7/50
5/5 [==============================] - 6s 1s/step - loss: 0.0566 - accuracy: 0.9935
Epoch 8/50
5/5 [==============================] - 6s 1s/step - loss: 0.0538 - accuracy: 0.9935
Epoch 9/50
5/5 [==============================] - 6s 1s/step - loss: 0.0521 - accuracy: 0.9935
Epoch 10/50
5/5 [==============================] - 6s 1s/step - loss: 0.0447 - accuracy: 0.9935
Epoch 11/50
5/5 [==============================] 